## Graph programs

The Graph Programs are a special data type representing a workflow of actions and decisions with calls to other programs. They are used by our own custom Agent, the `GraphProgramInterpreter`. In order help you to build them, we provide two ways of doing it: Using Python or Cypher.

In [1]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
	name = "main",
	description = "The main program",
)

main.add("answer", gp.Action(
	tool = "Speak",
	purpose = "Answer the user question",
	prompt = "Please answer to the following question: {{objective}}",
	inputs = ["objective"],
	ouput = "answer",
))

main.connect("start", "answer")
main.connect("answer", "end")

# We build the program to verify its structure
main.build()

# We can convert it to the cypher representation
print(main.to_cypher())

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


// @desc: The main program
CREATE
// Nodes declaration
(start:Control {purpose: "Start"}),
(end:Control {purpose: "End"}),
(answer:Action {
  purpose: "Answer the user question",
  tool: "Speak",
  prompt: "Please answer to the following question: {{objective}}",
  inputs: [
    "objective"
  ]
}),
// Structure declaration
(start)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)


In [2]:
# Or we can visualize it using pyvis

# Use notebook=False if you are running in a local settings and want to open a browser page
main.show(notebook=False)

main.html


### Using decision-making steps

Decision making steps allow the model to branch over different paths in a program, like conditions in traditional programming, it allow conditional loops and multi-output decisions.

In [3]:
main = gp.GraphProgram(
    name="main",
    description="The main program",
)
    
main.add("is_objective_unclear", gp.Decision(
    purpose="Check if the question needs clarification or not",
    prompt="Is the following question unclear?\n{{question}}",
    inputs=["objective"],
))

main.add("clarify", gp.Action(
    purpose="Ask one question to clarify the user's objective",
    tool="AskUser",
    prompt="Please pick one question to clarify the following: {{objective}}",
    inputs=["objective"],
    output="clarification"
))

main.add("answer", gp.Action(
    purpose="Answer the question",
    tool="Speak",
    prompt="Please answer to the following question: {{objective}}",
    inputs=["objective"],
))
    
main.add("refine_objective", gp.Action(
    purpose="Refine the objective",
    tool="Predict",
    prompt= \
"""You asked the following question:
Question: {{clarification}}

Please refine the following objective:
Objective: {{objective}}""",
    inputs=["objective", "clarification"],
    output="objective"
))
    
main.connect("start", "is_objective_unclear")
main.connect("is_objective_unclear", "clarify", label="Clarify")
main.connect("is_objective_unclear", "answer", label="Answer")
main.connect("clarify", "refine_objective")
main.connect("refine_objective", "answer")
main.connect("answer", "end")

main.build()

print(main.to_cypher())

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {purpose: "Start"}),
(end:Control {purpose: "End"}),
(is_objective_unclear:Decision {
  purpose: "Check if the question needs clarification or not",
  prompt: "Is the following question unclear?\n{{question}}",
  inputs: [
    "objective"
  ]
}),
(clarify:Action {
  purpose: "Ask one question to clarify the user's objective",
  tool: "AskUser",
  prompt: "Please pick one question to clarify the following: {{objective}}",
  inputs: [
    "objective"
  ],
  output: "clarification"
}),
(answer:Action {
  purpose: "Answer the question",
  tool: "Speak",
  prompt: "Please answer to the following question: {{objective}}",
  inputs: [
    "objective"
  ]
}),
(refine_objective:Action {
  purpose: "Refine the objective",
  tool: "Predict",
  prompt: "You asked the following question:\nQuestion: {{clarification}}\n\nPlease refine the following objective:\nObjective: {{objective}}",
  inputs: [
    "objective",
    "clarificati

In [4]:
# Or we can check it using pyvis like above

main.show()

main.html


#### Using Program calls

Using sub-programs is an important feature of HybridAGI, their are used to allow the graph programs to scale the numerous steps for long tasks, help the developer encapsulate behaviors and allow the system to adapt and learn by being able to dynamically call them or modify them.

In this example 

In [5]:

clarify_objective = gp.GraphProgram(
    name="clarify_objective",
    description="Clarify the objective by asking question to the user",
)

clarify_objective.add("is_anything_unclear", gp.Decision(
    purpose = "Check if the question is unclear",
    prompt = "Is the following question '{{objective}}' still unclear?",
    inputs = ["objective"],
))

clarify_objective.add("ask_question", gp.Action(
    purpose = "Ask question to clarify the user request",
    tool = "AskUser",
    prompt = "Pick one question to clarify the following user request: {{objective}}",
    inputs = ["objective"],
    output = "clarification",
))

clarify_objective.add("refine_objective", gp.Action(
    purpose = "Refine the question",
    tool = "Predict",
    prompt = "Refine the following '{{question}}' based on the clarfication step: {{clarification}}",
    inputs = ["objective", "clarification"],
    output = "objective",
))

clarify_objective.connect("start", "is_anything_unclear")
clarify_objective.connect("ask_question", "refine_objective")
clarify_objective.connect("is_anything_unclear", "ask_question", label="Clarify")
clarify_objective.connect("is_anything_unclear", "end", label="Answer")
clarify_objective.connect("refine_objective", "end")

clarify_objective.build()

main = gp.GraphProgram(
    name="main",
    description="The main program",
)

main.add("clarify_objective", gp.Program(
    purpose = "Clarify the user objective if needed",
    prompt = "{{objective}}",
    inputs = ["objective"],
    program = "clarify_objective"
))

main.add("answer", gp.Action(
    purpose = "Answer the objective's question",
    tool = "Speak",
    prompt = "Answer the following question: {{objective}}",
))

main.connect("start", "clarify_objective")
main.connect("clarify_objective", "answer")
main.connect("answer", "end")

main.build()

print(main.to_cypher())

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {purpose: "Start"}),
(end:Control {purpose: "End"}),
(clarify_objective:Program {
  purpose: "Clarify the user objective if needed",
  program: "clarify_objective",
  prompt: "{{objective}}",
  inputs: [
    "objective"
  ]
}),
(answer:Action {
  purpose: "Answer the objective's question",
  tool: "Speak",
  prompt: "Answer the following question: {{objective}}"
}),
// Structure declaration
(start)-[:NEXT]->(clarify_objective),
(clarify_objective)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)


#### Note on graph program validation

Although we verify the structure of the program, we cannot confirm if the tool used is accurate or if the program referenced is correct outside of the execution environment. This implies that you should be cautious in using the appropriate names, otherwise, the interpreter Agent will generate an error when it encounters the problematic step.

#### Loading from Cypher

You can also load your programs from cypher, 